In [29]:
import pandas as pd

In [30]:
df = pd.read_csv("/data/raw_stock_ind.csv")

In [34]:
import json
string = df.head(1).info_json[0]
pd.json_normalize(df.head(1).info_json[0])

KeyError: 1

In [35]:
string = string.replace("\'", "\"")
student_details = json.loads(string)

TypeError: the JSON object must be str, bytes or bytearray, not Series

In [36]:
print(string)

0    {'address1': 'L&T House', 'address2': 'Ballard...
Name: info_json, dtype: object
